# Dashboard for DELFIN

Generate the CONTROL-file for DELFIN and submit jobs to bwUniCluster.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import py3Dmol
import os
import subprocess
import shutil
import re
from pathlib import Path

# Pfade dynamisch ermitteln
HOME = Path.home()
BASE_DIR = HOME / "calc"  # Benutzer-spezifisches Arbeitsverzeichnis

# Notebook-Verzeichnis ermitteln (wo dieses Notebook liegt)
# In Jupyter: nutze das aktuelle Arbeitsverzeichnis oder finde den Pfad über IPython
try:
    # Versuche den Notebook-Pfad über IPython zu ermitteln
    import IPython
    NOTEBOOK_DIR = Path(IPython.extract_module_locals()[1]['__vsc_ipynb_file__']).parent
except:
    # Fallback: aktuelles Arbeitsverzeichnis (funktioniert wenn Notebook dort geöffnet wird)
    NOTEBOOK_DIR = Path.cwd()

# SUBMIT_TEMPLATES_DIR suchen - verschiedene moegliche Pfade
_submit_candidates = [
    HOME / "software" / "submit_sh",
    NOTEBOOK_DIR / "submit_sh",
]
SUBMIT_TEMPLATES_DIR = None
for _path in _submit_candidates:
    if _path.exists():
        SUBMIT_TEMPLATES_DIR = _path
        break
if SUBMIT_TEMPLATES_DIR is None:
    SUBMIT_TEMPLATES_DIR = _submit_candidates[0]  # Default fuer Fehlermeldung

# ONLY_GOAT Template suchen - verschiedene moegliche Pfade
_goat_candidates = [
    HOME / "software" / "CONTROL_Templates" / "only_GOAT.txt",
    HOME / "CONTROL_Templates" / "only_GOAT.txt",
    NOTEBOOK_DIR / "CONTROL_Templates" / "only_GOAT.txt",
]
ONLY_GOAT_TEMPLATE_PATH = None
for _path in _goat_candidates:
    if _path.exists():
        ONLY_GOAT_TEMPLATE_PATH = _path
        break
if ONLY_GOAT_TEMPLATE_PATH is None:
    ONLY_GOAT_TEMPLATE_PATH = _goat_candidates[0]  # Default fuer Fehlermeldung

# Mapping von Job-Typ zu Template-Datei
JOB_TYPE_TEMPLATES = {
    'Short (24h)': 'submit_short.sh',
    'Standard (48h)': 'submit_standard.sh',
    'Long (72h)': 'submit_long.sh',
    'Extra Long (120h)': 'submit_extralong.sh'
}

def is_smiles(text):
    """
    Prueft ob der Text ein SMILES-String ist.
    SMILES enthalten typisch: C, N, O, S, P, F, Cl, Br, I, =, #, (, ), [, ], @, +, -, Zahlen
    """
    text = text.strip()
    # Einzelne Zeile ohne Leerzeichen (ausser in eckigen Klammern)
    if '\n' in text:
        return False
    # SMILES Pattern: Buchstaben, Zahlen, und typische SMILES-Zeichen
    smiles_pattern = r'^[A-Za-z0-9\[\]()=#@+\-\\\/.:]+$'
    return bool(re.match(smiles_pattern, text))

def clean_input_data(input_text):
    """
    Verarbeitet Eingabedaten: SMILES oder XYZ-Koordinaten.
    
    - SMILES: wird direkt zurueckgegeben
    - XYZ mit Header: Header (Atomanzahl + Kommentar) wird entfernt
    - XYZ ohne Header: wird direkt zurueckgegeben
    
    Returns: (cleaned_text, input_type)
    input_type: 'smiles' oder 'xyz'
    """
    text = input_text.strip()
    
    if not text:
        return '', 'empty'
    
    # Pruefe auf SMILES
    if is_smiles(text):
        return text, 'smiles'
    
    # XYZ-Koordinaten verarbeiten
    lines = text.split('\n')
    if len(lines) < 2:
        return text, 'xyz'
    
    # Pruefe ob erste Zeile nur eine Zahl ist (Atomanzahl)
    first_line = lines[0].strip()
    try:
        atom_count = int(first_line)
        # Erste Zeile ist eine Zahl -> XYZ-Header vorhanden
        # Entferne erste zwei Zeilen (Atomanzahl + Kommentar)
        cleaned_lines = lines[2:]
        return '\n'.join(cleaned_lines).strip(), 'xyz'
    except ValueError:
        # Keine Zahl -> kein XYZ-Header, Koordinaten direkt
        return text, 'xyz'

# Vollstaendiges CONTROL.txt Template aus define.py (angepasst fuer bwUniCluster)
DEFAULT_CONTROL = """input_file=input.txt
NAME=
SMILES=
charge=[CHARGE]
------------------------------------
Solvation:
implicit_solvation_model=CPCM
solvent=[SOLVENT]
XTB_SOLVATOR=no
number_explicit_solv_molecules=2
------------------------------------
Global geometry optimisation:
xTB_method=XTB2
XTB_OPT=no
XTB_GOAT=no
CREST=no
multiplicity_global_opt=
------------------------------------
IMAG=yes
IMAG_scope=initial
IMAG_option=2
allow_imaginary_freq=0
IMAG_sp_energy_window=1e-3
IMAG_optimize_candidates=no
------------------------------------
calc_prop_of_interest=no
properties_of_interest=IP,EA
------------------------------------
Redox steps:
calc_initial=yes
oxidation_steps=1,2,3
reduction_steps=1,2,3
method=classic|manually|OCCUPIER
calc_potential_method=2
------------------------------------
ESD module (excited state dynamics):
ESD_modul=no
ESD_modus=TDDFT|deltaSCF|hybrid1
ESD_frequency=yes
states=S1,T1,S2,T2
ISCs=S1>T1,T1>S1
ICs=S2>S1
emission_rates=f,p
phosp_IROOT=1,2,3
phosp_keywords=
fluor_keywords=
TROOTSSL=-1,0,1
addition_S0=
DOHT=TRUE
ESD_LINES=LORENTZ
ESD_LINEW=50
ESD_INLINEW=250
ESD_NPOINTS=131072
ESD_MAXTIME=12000
hybrid1_geom_MaxIter=60
--------------------
Electrical Properties:
elprop_Dipole=no
elprop_Quadrupole=no
elprop_Hyperpol=no
elprop_Polar=no
elprop_PolarVelocity=no
elprop_PolarDipQuad=no
elprop_PolarQuadQuad=no
--------------------
deltaSCF Settings:
deltaSCF_DOMOM=true
deltaSCF_PMOM=true
deltaSCF_keepinitialref=true
deltaSCF_SOSCFHESSUP=LSR1
deltaSCF_keywords=FreezeAndRelease
deltaSCF_maxiter=300
deltaSCF_SOSCFConvFactor=500
deltaSCF_SOSCFMaxStep=0.1
--------------------
TDDFT Settings:
TDDFT_TDDFT_maxiter=500
TDDFT_nroots=15
TDDFT_maxdim=30
TDDFT_TDA=FALSE
TDDFT_followiroot=true
TDDFT_SOC=false
------------------------------------
MANUALLY:
multiplicity_0=
additions_0=
additions_TDDFT=
additions_T1=
additions_S1=
multiplicity_ox1=
additions_ox1=
multiplicity_ox2=
additions_ox2=
multiplicity_ox3=
additions_ox3=
multiplicity_red1=
additions_red1=
multiplicity_red2=
additions_red2=
multiplicity_red3=
additions_red3=
------------------------------------
Level of Theory:
functional=PBE0
disp_corr=D4
ri_jkx=RIJCOSX
relativity=ZORA
aux_jk=def2/J
aux_jk_rel=SARC/J
main_basisset=def2-SVP
main_basisset_rel=ZORA-def2-SVP
metal_basisset=def2-TZVP
metal_basisset_rel=SARC-ZORA-TZVP
first_coordination_sphere_metal_basisset=no
first_coordination_sphere_scale=1.3
geom_opt=OPT
freq_type=FREQ
initial_guess=PModel
temperature=298.15
maxiter=125
qmmm_option=QM/PBEH-3c
------------------------------------
Reference value:
E_ref=
------------------------------------
Literature_reference=
reference_CV=V Vs. Fc+/Fc
E_00_exp=
E_red_exp=
E_red_2_exp=
E_red_3_exp=
E_ox_exp=
E_ox_2_exp=
E_ox_3_exp=
*E_red_exp=
*E_ox_exp=
------------------------------------
Prints:
print_MOs=no
print_Loewdin_population_analysis=no
------------------------------------
Resource Settings:
PAL=40
maxcore=6000
parallel_workflows=yes
pal_jobs=4
orca_parallel_strategy=auto
enable_job_timeouts=no
job_timeout_hours=36
opt_timeout_hours=14
frequency_timeout_hours=36
sp_timeout_hours=3
------------------------------------
Automatic Error Recovery & Retry:
enable_auto_recovery=yes
max_recovery_attempts=1
enable_adaptive_parallelism=yes
enable_performance_metrics=yes
------------------------------------
OCCUPIER-Settings:
--------------------
OCCUPIER_method=auto
OCCUPIER_tree=own
OWN_TREE_PURE_WINDOW=3
OWN_progressive_from=no
fob_equal_weights=yes
frequency_calculation_OCCUPIER=no
occupier_selection=tolerance
occupier_precision=3
occupier_epsilon=5e-4
clean_override_window_h=0.002
clean_quality_improvement=0.05
clean_quality_good=0.05
maxiter_occupier=125
geom_opt_OCCUPIER=OPT
pass_wavefunction=no
approximate_spin_projection_APMethod=2
--------------------
OCCUPIER_sequence_profiles:
-3,-2,-1,0,+1,+2,+3=[
even electron number:
even_seq = [
  {"index": 1, "m": 1, "BS": "",    "from": 0},
  {"index": 2, "m": 1, "BS": "1,1", "from": 1},
  {"index": 3, "m": 1, "BS": "2,2", "from": 2},
  {"index": 4, "m": 3, "BS": "",    "from": 1},
  {"index": 5, "m": 3, "BS": "3,1", "from": 4},
  {"index": 6, "m": 3, "BS": "4,2", "from": 5},
  {"index": 7, "m": 5, "BS": "",    "from": 4}
]
-------------------
odd electron number:
odd_seq = [
  {"index": 1, "m": 2, "BS": "",    "from": 0},
  {"index": 2, "m": 2, "BS": "2,1", "from": 1},
  {"index": 3, "m": 2, "BS": "3,2", "from": 2},
  {"index": 4, "m": 4, "BS": "",    "from": 1},
  {"index": 5, "m": 4, "BS": "4,1", "from": 4},
  {"index": 6, "m": 4, "BS": "5,2", "from": 5},
  {"index": 7, "m": 6, "BS": "",    "from": 4}
]
]
"""

common_layout = widgets.Layout(width='500px')
common_style = {'description_width': 'initial'}

# === Widgets ===
job_name_widget = widgets.Text(
    value='',
    placeholder='z.B. Fe_Complex_Ox',
    description='Job Name:',
    layout=common_layout,
    style=common_style
)

job_type_widget = widgets.RadioButtons(
    options=['Short (24h)', 'Standard (48h)', 'Long (72h)', 'Extra Long (120h)'],
    value='Standard (48h)',
    description='Job Type:',
    style=common_style,
    layout=widgets.Layout(width='500px')
)

coords_help = widgets.Label("Input: XYZ coordinates (header auto-removed) or SMILES string")
coords_widget = widgets.Textarea(
    value='',
    placeholder='Paste XYZ coordinates or SMILES:\n\nXYZ example:\n42\nComment\nFe  0.0  0.0  0.0\nC   1.5  0.0  0.0\n\nSMILES example:\nCCO or c1ccccc1',
    description='Input:',
    layout=widgets.Layout(width='500px', height='200px'),
    style=common_style
)

control_help = widgets.Label("CONTROL.txt - edit parameters as needed")
control_widget = widgets.Textarea(
    value=DEFAULT_CONTROL,
    description='CONTROL.txt:',
    layout=widgets.Layout(width='500px', height='500px'),
    style=common_style
)

# Submit Button - BLAU
submit_button = widgets.Button(
    description='SUBMIT JOB',
    button_style='primary',
    layout=widgets.Layout(width='200px', height='50px')
)

output_area = widgets.Output()

# Molecule visualization
mol_output = widgets.Output(layout=widgets.Layout(
    border='2px solid #1976d2',
    width='600px',
    height='450px',
    overflow='hidden'
))

only_goat_label = widgets.Label("Only GOAT:")
only_goat_charge = widgets.IntText(
    value=0,
    description='Charge:',
    style=common_style,
    layout=widgets.Layout(width='160px')
)
only_goat_solvent = widgets.Text(
    value='',
    placeholder='e.g. water',
    description='Solvent:',
    style=common_style,
    layout=widgets.Layout(width='220px')
)
only_goat_submit_button = widgets.Button(
    description='SUBMIT ONLY GOAT',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)
only_goat_output = widgets.Output()

def update_molecule_view(change=None):
    with mol_output:
        clear_output()
        raw_input = coords_widget.value.strip()
        if raw_input:
            cleaned_data, input_type = clean_input_data(raw_input)
            
            if input_type == 'smiles':
                # SMILES - zeige Text-Info (3D-Visualisierung braeuchte RDKit)
                print(f"SMILES detected: {cleaned_data}")
                print("\n(3D visualization requires coordinates)")
            else:
                # XYZ-Koordinaten visualisieren
                coords = cleaned_data
                lines = [l for l in coords.split('\n') if l.strip()]
                num_atoms = len(lines)
                xyz_data = f"{num_atoms}\nGenerated by widget\n{coords}"
                mol_view = py3Dmol.view(width=590, height=440)
                mol_view.addModel(xyz_data, "xyz")
                mol_view.setStyle({}, {"stick": {"radius": 0.15}, "sphere": {"scale": 0.22}})
                mol_view.zoomTo()
                mol_view.show()
        else:
            print("Please enter XYZ coordinates or SMILES.")

coords_widget.observe(update_molecule_view, names='value')

def reset_form():
    """Setzt das Formular zurueck nach erfolgreichem Submit."""
    job_name_widget.value = ''
    coords_widget.value = ''
    control_widget.value = DEFAULT_CONTROL
    job_type_widget.value = 'Standard (48h)'
    only_goat_charge.value = 0
    only_goat_solvent.value = ''
    # Molekuel-Ansicht leeren
    with mol_output:
        clear_output()
        print("Please enter XYZ coordinates or SMILES.")

def handle_submit(button):
    with output_area:
        clear_output()
        
        job_name = job_name_widget.value.strip()
        job_type = job_type_widget.value
        control_content = control_widget.value
        raw_input = coords_widget.value.strip()
        
        if not job_name:
            print("Error: Job name cannot be empty!")
            return
        
        if not raw_input:
            print("Error: Input (coordinates or SMILES) cannot be empty!")
            return
        
        # Input verarbeiten (SMILES oder XYZ)
        input_content, input_type = clean_input_data(raw_input)
        
        if not input_content:
            print("Error: No valid input found!")
            return
        
        safe_job_name = "".join(c for c in job_name if c.isalnum() or c in ('_', '-'))
        if not safe_job_name:
            print("Error: Job name contains only invalid characters!")
            return
        
        job_dir = BASE_DIR / safe_job_name
        
        # Template-Datei fuer den Job-Typ ermitteln
        template_filename = JOB_TYPE_TEMPLATES.get(job_type, 'submit_standard.sh')
        template_path = SUBMIT_TEMPLATES_DIR / template_filename
        
        if not template_path.exists():
            print(f"Error: Template not found: {template_path}")
            print(f"Please create the template file for job type '{job_type}'")
            return
        
        try:
            job_dir.mkdir(parents=True, exist_ok=True)
            
            # CONTROL.txt schreiben
            control_path = job_dir / "CONTROL.txt"
            control_path.write_text(control_content)
            
            # input.txt schreiben (SMILES oder XYZ ohne Header)
            input_path = job_dir / "input.txt"
            input_path.write_text(input_content)
            
            # Submit-Script aus Template lesen und {job_name} ersetzen
            template_content = template_path.read_text()
            submit_script = template_content.replace('{job_name}', safe_job_name)
            
            submit_path = job_dir / "submit_job.sh"
            submit_path.write_text(submit_script)
            submit_path.chmod(0o755)
            
            # Job einreichen
            result = subprocess.run(
                ['sbatch', 'submit_job.sh'],
                cwd=job_dir,
                capture_output=True,
                text=True
            )
            
            if result.returncode == 0:
                job_id = result.stdout.strip().split()[-1]
                print(f"Job successfully submitted!")
                print(f"Job ID: {job_id}")
                print(f"Job Type: {job_type}")
                print(f"Input Type: {input_type.upper()}")
                print(f"Directory: {job_dir}")
                print(f"Template: {template_filename}")
                print(f"")
                print(f"Check status: squeue -u $USER")
                print(f"View output: tail -f {job_dir}/delfin_*.out")
                
                # Formular zuruecksetzen nach erfolgreichem Submit
                reset_form()
            else:
                print(f"Error submitting job:")
                print(result.stderr)
        
        except Exception as e:
            print(f"Error creating job: {str(e)}")

def handle_only_goat_submit(button):
    with only_goat_output:
        clear_output()

        job_name = job_name_widget.value.strip()
        job_type = job_type_widget.value
        raw_input = coords_widget.value.strip()
        charge_value = only_goat_charge.value
        solvent_value = only_goat_solvent.value.strip()

        if not job_name:
            print("Error: Job name cannot be empty!")
            return

        if not raw_input:
            print("Error: Input (coordinates or SMILES) cannot be empty!")
            return

        if solvent_value == '':
            print("Error: Solvent cannot be empty!")
            return

        # Input verarbeiten (SMILES oder XYZ)
        input_content, input_type = clean_input_data(raw_input)

        if not input_content:
            print("Error: No valid input found!")
            return

        safe_job_name = "".join(c for c in job_name if c.isalnum() or c in ('_', '-'))
        if not safe_job_name:
            print("Error: Job name contains only invalid characters!")
            return

        if not ONLY_GOAT_TEMPLATE_PATH.exists():
            print(f"Error: Only GOAT template not found: {ONLY_GOAT_TEMPLATE_PATH}")
            return

        job_dir = BASE_DIR / safe_job_name

        # Template-Datei fuer den Job-Typ ermitteln
        template_filename = JOB_TYPE_TEMPLATES.get(job_type, 'submit_standard.sh')
        template_path = SUBMIT_TEMPLATES_DIR / template_filename

        if not template_path.exists():
            print(f"Error: Template not found: {template_path}")
            print(f"Please create the template file for job type '{job_type}'")
            return

        try:
            job_dir.mkdir(parents=True, exist_ok=True)

            # CONTROL.txt aus Only GOAT Template schreiben
            control_template = ONLY_GOAT_TEMPLATE_PATH.read_text()
            control_content = (control_template
                .replace('[CHARGE]', str(charge_value))
                .replace('[SOLVENT]', solvent_value)
            )
            control_path = job_dir / "CONTROL.txt"
            control_path.write_text(control_content)

            # input.txt schreiben (SMILES oder XYZ ohne Header)
            input_path = job_dir / "input.txt"
            input_path.write_text(input_content)

            # Submit-Script aus Template lesen und {job_name} ersetzen
            template_content = template_path.read_text()
            submit_script = template_content.replace('{job_name}', safe_job_name)

            submit_path = job_dir / "submit_job.sh"
            submit_path.write_text(submit_script)
            submit_path.chmod(0o755)

            # Job einreichen
            result = subprocess.run(
                ['sbatch', 'submit_job.sh'],
                cwd=job_dir,
                capture_output=True,
                text=True
            )

            if result.returncode == 0:
                job_id = result.stdout.strip().split()[-1]
                print("Only GOAT job successfully submitted!")
                print(f"Job ID: {job_id}")
                print(f"Job Type: {job_type}")
                print(f"Input Type: {input_type.upper()}")
                print(f"Charge: {charge_value}")
                print(f"Solvent: {solvent_value}")
                print(f"Directory: {job_dir}")
                print(f"Template: {template_filename}")
                print(f"")
                print(f"Check status: squeue -u $USER")
                print(f"View output: tail -f {job_dir}/delfin_*.out")

                # Formular zuruecksetzen nach erfolgreichem Submit
                reset_form()
            else:
                print("Error submitting job:")
                print(result.stderr)

        except Exception as e:
            print(f"Error creating job: {str(e)}")

only_goat_submit_button.on_click(handle_only_goat_submit)

submit_button.on_click(handle_submit)

spacer = widgets.Label(value="", layout=widgets.Layout(height='20px'))

# === Layout ===
ui_left = widgets.VBox([
    job_name_widget,
    spacer,
    job_type_widget,
    spacer,
    coords_help,
    coords_widget,
    spacer,
    control_help,
    control_widget,
    spacer,
    submit_button,
    output_area
], layout=widgets.Layout(width='55%', padding='10px'))

ui_right = widgets.VBox([
    widgets.Label("Molecule visualization:"),
    mol_output,
    spacer,
    only_goat_label,
    widgets.HBox([
        only_goat_charge,
        only_goat_solvent,
        only_goat_submit_button
    ], layout=widgets.Layout(width='100%')),
    only_goat_output
], layout=widgets.Layout(width='45%', padding='10px'))

full_ui = widgets.HBox([ui_left, ui_right])
display(full_ui)

## Job-Status

In [21]:
!squeue -u $USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           2921236       cpu    Test3 ka_ew740 PD       0:00      1 (Resources)
           2919093       cpu delfin_t ka_ew740  R      27:22      1 uc3n102


In [20]:
!squeue -u $USER --start

             JOBID PARTITION     NAME     USER ST          START_TIME  NODES SCHEDNODES           NODELIST(REASON)
           2921236       cpu    Test3 ka_ew740 PD 2026-01-14T16:48:17      1 uc3n038              (Resources)


In [19]:
!ls -lht ~/calc/

total 12K
drwxr-xr-x 2 ka_ew7404 ka_ibcs 4.0K Jan 14 13:29 Test3
drwxr-xr-x 2 ka_ew7404 ka_ibcs 4.0K Jan 14 13:04 Test2
drwxr-xr-x 4 ka_ew7404 ka_ibcs 4.0K Jan 14 11:01 Test1
